# ROI Stand Alone Service

# 0. Imports

1. Verander **git_dir** in de map op jouw computer met de code. <br>
2. Run vervolgens de cell door met de **Run** knop bovenin het scherm of door **ctrl+enter** te drukken. <br>
Het bolletje rechts boven geeft aan of het notebook bezig is (naast Python 3 (ipykernel)). Als deze zwart gekleurd is, dan is het notebook bezig.<br>
Als er "Imports are succesfull!" onder de cell komt te staan na het uitvoeren, dan is deze cell succesvol uitgevoerd. <br>

In [ ]:
import os
import sys
import numpy as np
import shutil
%load_ext autoreload
%autoreload 2

# insert the directory where you extracted the zip file here:
git_dir = r"C:\Werk\Projecten\P1414_ROI\Github_P1414\P1414"

sys.path.append(os.path.join(git_dir, 'Code'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox/Readers'))

from notebooks.background_scripting.v1.test_connection import TestConnection
test_connection = TestConnection()

# 1. Model specifications 

## 1.1. Select basemodel
De widgets zijn alleen te zien als je het notebook zelf runt! In de documentatie zijn deze niet zichtbaar. <br>
1. Run de cell hieronder.<br>
2. Vul in de widget "path to folder with models" de map in waarin de gedownloade modellen staan. <br>
3. Vervolgens klik je update settings en kan je een model selecteren in de lijst. <br>
4. Je kan het scenario dat je wil door rekenen ook een naam geven, door deze in te vullen bij "Scenario name". <br>
5. Ten slotte klik je wederom op update settings.

In [ ]:
from notebooks.background_scripting.v1.select_basemodel import ModelSettings

model_settings = ModelSettings(git_dir)
model_settings.display_widgets()

## 1.2. Model settings
1. Run de cell hieronder.<br>
2. Kies de start- en stoptijd van het model. <br>
3. Kies de interval om resultaten weg te schrijven bij "Interval to write map file". <br>
4. Vul de locatie in van het D-HYDRO script _run_dimr.bat_. Dit scripts is te vinden in de map van je D-HYDRO installatie. De map is waarschijnlijk vergelijkbaar aan de standaard waarde in de widget "Location of dhydro run_dimr". <br>
5. Klik op "Update settings <br>

In [ ]:
from notebooks.background_scripting.v1.modify_mdu import ModifyMDU, copy_model

model_path = copy_model(os.path.join(model_settings.settings['folder'], model_settings.settings['model']), model_settings.settings['scenario_name'])
modify_mdu = ModifyMDU(model_path)
modify_mdu.display_widgets()

## 1.3.1. Dambreak location
Kies hieronder zelf een dambreak locatie. <br>
1. Run de cell hieronder.<br>
2. Kies de locatie van je damdoorbraak. Dit doe je door een lijn te trekken door een dam (rood) heen. Deze lijn moet ook een 1D2D link (groen) snijden. Het kan even duren voordat alle dammen en links zichtbaar zijn. <br>
3. Kies het bovenstroomse punt (punt in watergang) voor het water door damdoorbraak stroomt. <br>
4. Kies het benedenstroomse punt (grid cell) waar het water naar toe stroomt. <br>
5. Bekijk je geselecteerde locatie. Als je niet tevereden bent, run deze cell dan opnieuw.

In [ ]:
from notebooks.background_scripting.v1.modify_dambreak_v3 import DambreakWidget
from notebooks.background_scripting.v1.modify_dambreak_v3 import UseTemplateDambreak

# if you want, you can select a dambreach from the template by setting this to True. 
# You can add templates by adding a functioning dike breach location to \data\dambreak_database.json.
use_template = False 
if use_template == True:
    dambreak_template = UseTemplateDambreak(model_path)
    dambreak_template.draw_map()
if use_template == False:
    dambreakWidget = DambreakWidget(model_path)
    dambreakWidget.draw_map()

## 1.4. Dambreak settings
Specificeer de parameters van de Dambreak <br>
1. Run de cell hieronder.<br>
2. Vul alle velden in met de waardes die je wil.<br>
3. Klik op "update settings". 

In [ ]:
from notebooks.background_scripting.v1.modify_dambreak_v3 import ModifyDambreak

if use_template == False:
    add_dambreak = ModifyDambreak(model_path, dambreakWidget.settings, dambreakWidget.keringen, False, None)
else:
    add_dambreak = ModifyDambreak(model_path, None, None, True, dambreak_template.kering_choice)
add_dambreak.display_widgets()

## 1.5. Boundary conditions
Specificeer de randvoorwaarden van het model. <br>
1. Run de cell hieronder.<br>
2. Voer het waterlevel in de Noordzee in. <br>
3. Voer het waterlevel in het Markermeer in. <br>
4. Bepaal de afvoergolf in de Rijn, door de parameters aan te passen.<br>
5. Klik op "update settings", je ziet dan de huidige instellingen. Je kan dingen nu verder aanpassen, en opnieuw op "update settings" klikken, totdat je tevreden bent.


In [ ]:
from notebooks.background_scripting.v1.modify_boundaries import ModifyBoundaries, ModifyRhineDischarge

modify_boundaries = ModifyBoundaries(model_path, modify_mdu.settings, add_dambreak.dambreak_settings)
modify_boundaries.display_widgets()

# 2. Run Model
Voer een simulatie uit. <br>
1. Run de cell hieronder. <br>
2. Klik op "Run model" <br>
3. Het model begint nu. Eerst moet het model initialiseren, dit kan een tijdje duren. Hierna zal het progressie balkje vol lopen. 

In [ ]:
from notebooks.background_scripting.v1.model_runner import ModelRunner

model_runner = ModelRunner(model_path)
model_runner.display_widgets()

# 3. Post processing

## 3.1. Plot map
Visualiseer de resultaten op kaart. <br>
1. Run de cell hieronder.<br>
2. Voer de gewenste parameters in voor de plot. <br>
3. Klik op "Plot map". De eerste plot duurt iets langer.<br>
Indien je een ander model wil bekijken dan je hebt doorgerekend (of als je niks wil doorrekenen), dan kan je dat doen door in de cell hieronder een nieuw model_path toe te voegen. Voeg de volgende lijn toe: <br>
_model_path = r"PATH_TO_MODEL"_

In [ ]:
from notebooks.background_scripting.v1.post_processing import PlotSettingsMap, MapPlotter

plot_settings = PlotSettingsMap(model_path)
plot_settings.display_widgets()

## 3.2. Plot dambreak
Maak grafieken van de dambreak variabelen.

1. Run de cell hieronder.<br>
2. Voer de gewenste parameters in voor de plot. <br>
3. Klik op "Plot breach information".<br>
Indien je een ander model wil bekijken dan je hebt doorgerekend (of als je niks wil doorrekenen), dan kan je dat doen door in de cell hieronder een nieuw model_path toe te voegen. Voeg de volgende lijn toe: <br>
_model_path = r"PATH_TO_MODEL"_

In [ ]:
from notebooks.background_scripting.v1.post_processing import PlotSettingsBreach, BreachPlotter

plot_settings = PlotSettingsBreach(model_path)
plot_settings.display_widgets()